# Interactive Keypoint Comparison Tool

이 노트북은 기존 키포인트 데이터와 비디오에서 새로 추출한 키포인트를 **인터랙티브하게** 비교할 수 있는 GUI 도구입니다.

## 사용법
1. 데이터 경로 설정 후 셀 실행
2. **Prev/Next 버튼**으로 프레임 이동
3. **Extract 버튼**으로 현재 프레임의 키포인트 추출
4. **실시간 시각적 비교** 확인

## 색상 구분
- **빨간색**: 원본 Pose 키포인트
- **파란색**: 추출된 Pose 키포인트  
- **초록색**: 원본 Face 키포인트
- **청록색**: 추출된 Face 키포인트
- **주황색**: 원본 Hand 키포인트
- **노란색**: 추출된 Hand 키포인트

In [ ]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
from typing import Dict, List, Tuple

# matplotlib 설정 - 불필요한 plot 생성 방지
plt.ioff()  # 인터랙티브 모드 완전 끄기
import matplotlib
matplotlib.use('Agg')  # 백엔드를 Agg로 설정

# 키포인트 추출을 위한 import
try:
    import pyopenpose
    OPENPOSE_AVAILABLE = True
    print("✅ OpenPose available")
except ImportError as e:
    OPENPOSE_AVAILABLE = False
    print("❌ OpenPose not available", e)

try:
    import mediapipe as mp
    MEDIAPIPE_AVAILABLE = True
    print("✅ MediaPipe available")
except ImportError:
    MEDIAPIPE_AVAILABLE = False
    print("❌ MediaPipe not available")
# 프로젝트 모듈 import
try:
    from keypoint_extractor import KeypointExtractor
    print("✅ KeypointExtractor imported")
except ImportError as e:
    print(f"❌ KeypointExtractor import failed: {e}")

plt.style.use('default')

## ⚙️ 설정 및 초기화

In [ ]:
# 📁 데이터 경로 설정
import os
from pathlib import Path

# 현재 notebook 파일의 디렉토리 기준으로 상위 폴더의 data 경로 생성
current_dir = Path(os.getcwd())  # 현재 작업 디렉토리
notebook_dir = current_dir if current_dir.name == "utils" else current_dir / "utils"
project_root = notebook_dir.parent  # utils의 상위 폴더
DATA_ROOT = project_root / "data"

SPLIT_NAME = "train"  # train, val, test 중 선택
DATASET_NAME = "03_syn_word"  # 비교할 데이터셋 이름

# 경로 구성
keypoint_base_path = DATA_ROOT / SPLIT_NAME / 'keypoint' / f"{DATASET_NAME}_keypoint"
video_base_path = DATA_ROOT / SPLIT_NAME / 'video' / f"{DATASET_NAME}_video"

print(f"📂 Current working directory: {current_dir}")
print(f"📂 Project root: {project_root}")
print(f"📂 Data root: {DATA_ROOT}")
print(f"🔍 Keypoint path: {keypoint_base_path}")
print(f"🎬 Video path: {video_base_path}")
print(f"✅ Keypoint path exists: {keypoint_base_path.exists()}")
print(f"✅ Video path exists: {video_base_path.exists()}")

# 실제로 존재하는 비디오 파일만 찾기
available_videos = []
if keypoint_base_path.exists() and video_base_path.exists():
    keypoint_dirs = [d.name for d in keypoint_base_path.iterdir() if d.is_dir()]
    
    for keypoint_dir_name in keypoint_dirs:
        # 대응하는 .mp4 파일이 있는지 확인
        video_file = video_base_path / f"{keypoint_dir_name}.mp4"
        if video_file.exists():
            available_videos.append(f"{keypoint_dir_name}.mp4")
    
    print(f"\n📹 Found {len(available_videos)} available videos")
    if available_videos:
        print(f"   Examples: {available_videos[:3]}...")
else:
    print("❌ No videos found!")

## 🔧 핵심 함수 정의

In [ ]:
def load_keypoint_data(keypoint_file: Path) -> Dict:
    """키포인트 JSON 파일 로드"""
    with open(keypoint_file, 'r') as f:
        return json.load(f)

def extract_coordinates(keypoint_data: Dict) -> Dict[str, np.ndarray]:
    """키포인트 데이터에서 좌표 추출"""
    coords = {}
    
    if 'people' in keypoint_data:
        person = keypoint_data['people']
        
        # Pose keypoints
        if 'pose_keypoints_2d' in person:
            pose_data = person['pose_keypoints_2d']
            # 3차원 데이터인 경우 (x, y, confidence)
            if len(pose_data) == 75:  # 25 * 3
                coords['pose'] = np.array(pose_data).reshape(-1, 3)[:, :2]  # x, y만 사용
            # 2차원 데이터인 경우 (x, y)
            elif len(pose_data) == 50:  # 25 * 2
                coords['pose'] = np.array(pose_data).reshape(-1, 2)
        
        # Face keypoints
        if 'face_keypoints_2d' in person:
            face_data = person['face_keypoints_2d']
            # 3차원 데이터인 경우 (x, y, confidence)
            if len(face_data) == 210:  # 70 * 3
                coords['face'] = np.array(face_data).reshape(-1, 3)[:, :2]  # x, y만 사용
            # 2차원 데이터인 경우 (x, y)
            elif len(face_data) == 140:  # 70 * 2
                coords['face'] = np.array(face_data).reshape(-1, 2)
        
        # Hand keypoints
        if 'hand_left_keypoints_2d' in person:
            left_hand_data = person['hand_left_keypoints_2d']
            # 3차원 데이터인 경우 (x, y, confidence)
            if len(left_hand_data) == 63:  # 21 * 3
                coords['left_hand'] = np.array(left_hand_data).reshape(-1, 3)[:, :2]  # x, y만 사용
            # 2차원 데이터인 경우 (x, y)
            elif len(left_hand_data) == 42:  # 21 * 2
                coords['left_hand'] = np.array(left_hand_data).reshape(-1, 2)
        
        if 'hand_right_keypoints_2d' in person:
            right_hand_data = person['hand_right_keypoints_2d']
            # 3차원 데이터인 경우 (x, y, confidence)
            if len(right_hand_data) == 63:  # 21 * 3
                coords['right_hand'] = np.array(right_hand_data).reshape(-1, 3)[:, :2]  # x, y만 사용
            # 2차원 데이터인 경우 (x, y)
            elif len(right_hand_data) == 42:  # 21 * 2
                coords['right_hand'] = np.array(right_hand_data).reshape(-1, 2)
    
    return coords

def get_frame_from_video(video_path: Path, frame_idx: int, debug_func=None) -> np.ndarray:
    """비디오에서 특정 프레임 추출"""
    try:
        if debug_func:
            debug_func(f"🎬 Attempting to read frame {frame_idx} from {video_path}")
            debug_func(f"📁 Video file exists: {video_path.exists()}")
        
        if not video_path.exists():
            if debug_func:
                debug_func(f"❌ Video file does not exist: {video_path}")
            return None
        
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            if debug_func:
                debug_func(f"❌ Failed to open video: {video_path}")
            return None
            
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        if debug_func:
            debug_func(f"📊 Video info - Frames: {total_frames}, FPS: {fps}, Size: {width}x{height}")
        
        if frame_idx >= total_frames:
            if debug_func:
                debug_func(f"❌ Frame index {frame_idx} exceeds total frames {total_frames}")
            cap.release()
            return None
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        cap.release()
        
        if ret:
            if debug_func:
                debug_func(f"✅ Successfully read frame {frame_idx} with shape {frame.shape}")
            return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            if debug_func:
                debug_func(f"❌ Failed to read frame {frame_idx} - ret={ret}")
            return None
            
    except Exception as e:
        if debug_func:
            debug_func(f"❌ Exception in get_frame_from_video: {e}")
        return None

def extract_single_frame_keypoints(frame: np.ndarray, extractor) -> Dict:
    """단일 프레임에서 키포인트 추출"""
    if extractor.method == "mediapipe":
        return extractor._extract_mediapipe(frame)
    elif extractor.method == "openpose":
        return extractor._extract_openpose(frame)
    else:
        return {'people': {}}

def match_keypoints_by_position(orig_coords: np.ndarray, extr_coords: np.ndarray, max_distance: float = 50.0) -> Tuple[np.ndarray, np.ndarray]:
    """
    위치 기반으로 키포인트 매칭 - 가장 가까운 점끼리 연결
    원본 25개와 추출 15개의 정확한 매칭을 위함
    """
    if len(orig_coords) == 0 or len(extr_coords) == 0:
        return np.array([]), np.array([])
    
    matched_orig = []
    matched_extr = []
    used_extr_indices = set()
    
    # 각 원본 포인트에 대해 가장 가까운 추출 포인트 찾기
    for orig_point in orig_coords:
        best_distance = float('inf')
        best_extr_idx = -1
        
        for extr_idx, extr_point in enumerate(extr_coords):
            if extr_idx in used_extr_indices:
                continue
                
            distance = np.linalg.norm(orig_point - extr_point)
            if distance < best_distance and distance <= max_distance:
                best_distance = distance
                best_extr_idx = extr_idx
        
        # 매칭된 포인트가 있으면 추가
        if best_extr_idx != -1:
            matched_orig.append(orig_point)
            matched_extr.append(extr_coords[best_extr_idx])
            used_extr_indices.add(best_extr_idx)
    
    return np.array(matched_orig), np.array(matched_extr)

def calculate_part_stats(original_coords: Dict, extracted_coords: Dict) -> Dict[str, Dict]:
    """파트별 통계 계산 - 정확한 포인트 매칭으로 과장 방지"""
    part_stats = {}
    
    for part_name in ['pose', 'face', 'left_hand', 'right_hand']:
        part_stats[part_name] = {
            'detected_orig': False,
            'detected_extr': False,
            'mean_distance': 0.0,
            'mean_x_diff': 0.0,
            'mean_y_diff': 0.0,
            'valid_points_orig': 0,
            'valid_points_extr': 0,
            'matched_points': 0,  # 실제 매칭된 포인트 수
            'total_points': 0
        }
        
        if part_name in original_coords and part_name in extracted_coords:
            orig = original_coords[part_name]
            extr = extracted_coords[part_name]
            
            # 유효한 키포인트만 추출
            orig_valid = orig[np.any(orig != 0, axis=1)]
            extr_valid = extr[np.any(extr != 0, axis=1)]
            
            part_stats[part_name]['detected_orig'] = len(orig_valid) > 0
            part_stats[part_name]['detected_extr'] = len(extr_valid) > 0
            part_stats[part_name]['valid_points_orig'] = len(orig_valid)
            part_stats[part_name]['valid_points_extr'] = len(extr_valid)
            part_stats[part_name]['total_points'] = len(orig)
            
            if len(orig_valid) > 0 and len(extr_valid) > 0:
                # 위치 기반 정확한 매칭
                matched_orig, matched_extr = match_keypoints_by_position(orig_valid, extr_valid)
                
                if len(matched_orig) > 0:
                    part_stats[part_name]['matched_points'] = len(matched_orig)
                    
                    diff = matched_orig - matched_extr
                    distance = np.linalg.norm(diff, axis=1)
                    
                    part_stats[part_name]['mean_distance'] = float(np.mean(distance))
                    part_stats[part_name]['mean_x_diff'] = float(np.mean(diff[:, 0]))
                    part_stats[part_name]['mean_y_diff'] = float(np.mean(diff[:, 1]))
    
    return part_stats

def calculate_interval_stats(all_part_stats: List[Dict]) -> Dict:
    """구간 통계 계산 및 프레임 간 일관성 평가"""
    interval_stats = {}
    
    part_names = ['pose', 'face', 'left_hand', 'right_hand']
    
    for part_name in part_names:
        # 프레임별 평균 거리 차이 수집
        frame_distances = []
        x_diffs = []
        y_diffs = []
        detection_rates = {'orig': 0, 'extr': 0}
        matched_points_list = []
        
        valid_frames = 0
        
        for frame_stats in all_part_stats:
            if part_name in frame_stats:
                stats = frame_stats[part_name]
                if stats['matched_points'] > 0:  # 매칭된 포인트가 있을 때만
                    frame_distances.append(stats['mean_distance'])
                    x_diffs.append(stats['mean_x_diff'])
                    y_diffs.append(stats['mean_y_diff'])
                    matched_points_list.append(stats['matched_points'])
                    valid_frames += 1
                
                if stats['detected_orig']:
                    detection_rates['orig'] += 1
                if stats['detected_extr']:
                    detection_rates['extr'] += 1
        
        total_frames = len(all_part_stats)
        
        if frame_distances:
            # 프레임 간 거리 차이의 표준편차 계산
            frame_distance_std = float(np.std(frame_distances))
            mean_frame_distance = float(np.mean(frame_distances))
            mean_matched_points = float(np.mean(matched_points_list))
            
            # 일관성 점수 계산
            if mean_frame_distance > 0:
                coefficient_of_variation = frame_distance_std / mean_frame_distance
                consistency_score = 1.0 / (1.0 + coefficient_of_variation)
            else:
                consistency_score = 1.0
            
            interval_stats[part_name] = {
                'mean_distance': mean_frame_distance,
                'std_distance': frame_distance_std,
                'mean_x_diff': float(np.mean(x_diffs)),
                'std_x_diff': float(np.std(x_diffs)),
                'mean_y_diff': float(np.mean(y_diffs)),
                'std_y_diff': float(np.std(y_diffs)),
                'detection_rate_orig': detection_rates['orig'] / total_frames,
                'detection_rate_extr': detection_rates['extr'] / total_frames,
                'consistency_score': consistency_score,
                'frame_distance_variability': frame_distance_std,
                'coefficient_of_variation': coefficient_of_variation if mean_frame_distance > 0 else 0.0,
                'mean_matched_points': mean_matched_points,
                'valid_frames': valid_frames,
                'total_frames': total_frames
            }
        else:
            interval_stats[part_name] = {
                'mean_distance': 0.0,
                'std_distance': 0.0,
                'mean_x_diff': 0.0,
                'std_x_diff': 0.0,
                'mean_y_diff': 0.0,
                'std_y_diff': 0.0,
                'detection_rate_orig': detection_rates['orig'] / total_frames,
                'detection_rate_extr': detection_rates['extr'] / total_frames,
                'consistency_score': 0.0,
                'frame_distance_variability': 0.0,
                'coefficient_of_variation': 0.0,
                'mean_matched_points': 0.0,
                'valid_frames': 0,
                'total_frames': total_frames
            }
    
    return interval_stats

def format_part_stats_html(part_stats: Dict) -> str:
    """파트별 통계를 HTML로 포맷 - 매칭 정보 표시"""
    html = "<div style='font-family: monospace; font-size: 12px;'>"
    
    for part_name, stats in part_stats.items():
        part_display = part_name.replace('_', ' ').title()
        
        # 검출 상태
        orig_status = "✅" if stats['detected_orig'] else "❌"
        extr_status = "✅" if stats['detected_extr'] else "❌"
        
        # 색상 결정
        if stats['matched_points'] > 0:
            if stats['mean_distance'] < 10:
                color = "green"
                quality = "양호"
            elif stats['mean_distance'] < 20:
                color = "orange"
                quality = "주의"
            else:
                color = "red"
                quality = "보정필요"
        else:
            color = "gray"
            quality = "데이터없음"
        
        html += f"""
        <div style='border: 1px solid #ddd; margin: 5px 0; padding: 8px; background: #f9f9f9;'>
            <b style='color: {color};'>{part_display}</b><br>
            검출: 원본{orig_status}({stats['valid_points_orig']}) 추출{extr_status}({stats['valid_points_extr']})<br>
            매칭된 점: {stats['matched_points']}/{stats['total_points']}<br>
        """
        
        if stats['matched_points'] > 0:
            html += f"""
            평균거리: {stats['mean_distance']:.1f}px | X차이: {stats['mean_x_diff']:.1f}px | Y차이: {stats['mean_y_diff']:.1f}px<br>
            <span style='color: {color}; font-weight: bold;'>{quality}</span>
            """
        else:
            html += "<span style='color: gray;'>매칭 데이터 없음</span>"
        
        html += "</div>"
    
    html += "</div>"
    return html

def format_interval_stats_html(interval_stats: Dict, start_frame: int, end_frame: int) -> str:
    """구간 통계를 HTML로 포맷"""
    html = f"<div style='font-family: monospace; font-size: 12px;'>"
    html += f"<h3>📊 구간 분석 결과 (Frame {start_frame+1}-{end_frame+1})</h3>"
    
    for part_name, stats in interval_stats.items():
        part_display = part_name.replace('_', ' ').title()
        
        # 일관성 평가 기준
        consistency = stats['consistency_score']
        cv = stats['coefficient_of_variation']
        variability = stats['frame_distance_variability']
        
        if consistency > 0.85 and variability < 5:
            consistency_color = "green"
            consistency_text = "매우 일관적"
            action_text = "보정 불필요"
        elif consistency > 0.7 and variability < 15:
            consistency_color = "orange"
            consistency_text = "보통 일관적"
            action_text = "선택적 보정"
        else:
            consistency_color = "red"
            consistency_text = "불일치 심함"
            action_text = "보정 필요"
        
        # 검출률
        detection_orig = stats['detection_rate_orig'] * 100
        detection_extr = stats['detection_rate_extr'] * 100
        
        html += f"""
        <div style='border: 1px solid #ddd; margin: 8px 0; padding: 10px; background: #f9f9f9;'>
            <b style='font-size: 14px;'>{part_display}</b><br>
            <div style='margin: 5px 0;'>
                <b>검출률:</b> 원본 {detection_orig:.1f}% | 추출 {detection_extr:.1f}%<br>
                <b>평균 매칭점:</b> {stats['mean_matched_points']:.1f}개<br>
                <b>평균 거리차이:</b> {stats['mean_distance']:.1f}px<br>
                <b>프레임 간 변동성:</b> {variability:.1f}px (표준편차)<br>
                <b>변동계수:</b> {cv:.2f}<br>
                <b>일관성:</b> <span style='color: {consistency_color}; font-weight: bold;'>{consistency_text} ({consistency:.2f})</span><br>
                <b>권장사항:</b> <span style='color: {consistency_color}; font-weight: bold;'>{action_text}</span><br>
                <b>유효 프레임:</b> {stats['valid_frames']}/{stats['total_frames']}
            </div>
        </div>
        """
    
    # 전체 평가
    html += "<div style='border: 2px solid #333; margin: 10px 0; padding: 10px; background: #f0f0f0;'>"
    html += "<h4>🎯 전체 평가</h4>"
    
    all_consistency = [stats['consistency_score'] for stats in interval_stats.values() if stats['valid_frames'] > 0]
    all_variability = [stats['frame_distance_variability'] for stats in interval_stats.values() if stats['valid_frames'] > 0]
    
    if all_consistency:
        overall_consistency = np.mean(all_consistency)
        overall_variability = np.mean(all_variability)
        
        if overall_consistency > 0.8 and overall_variability < 10:
            html += "<p style='color: green; font-weight: bold;'>✅ 전체적으로 매우 일관적임. 보정 불필요</p>"
            html += f"<p style='color: green;'>프레임 간 차이가 안정적 (평균 변동: {overall_variability:.1f}px)</p>"
        elif overall_consistency > 0.7:
            html += "<p style='color: orange; font-weight: bold;'>⚠️ 보통 수준의 일관성. 선택적 보정 권장</p>"
            html += f"<p style='color: orange;'>프레임 간 차이에 약간의 변동 (평균 변동: {overall_variability:.1f}px)</p>"
        else:
            html += "<p style='color: red; font-weight: bold;'>❌ 심각한 불일치. 보정 필수</p>"
            html += f"<p style='color: red;'>프레임 간 차이가 매우 불안정 (평균 변동: {overall_variability:.1f}px)</p>"
        
        html += f"<p><b>전체 일관성 점수:</b> {overall_consistency:.2f}</p>"
        html += f"<p><b>전체 변동성:</b> {overall_variability:.1f}px</p>"
    else:
        html += "<p style='color: gray;'>분석할 데이터가 부족합니다.</p>"
    
    html += "</div></div>"
    
    return html

# 파일 기반 캐시 시스템 함수들
def get_cache_dir():
    """캐시 디렉토리 경로 반환"""
    cache_dir = Path("./keypoint_cache")
    cache_dir.mkdir(exist_ok=True)
    return cache_dir

def get_cache_filename(video_id: str, method: str, frame_idx: int) -> str:
    """캐시 파일명 생성"""
    safe_video_id = video_id.replace('.mp4', '').replace('/', '_')
    return f"{safe_video_id}_{method}_frame_{frame_idx:06d}.json"

def save_cache_to_file(video_id: str, method: str, frame_idx: int, data: Dict):
    """캐시를 파일로 저장"""
    cache_dir = get_cache_dir()
    filename = get_cache_filename(video_id, method, frame_idx)
    cache_file = cache_dir / filename
    
    try:
        with open(cache_file, 'w') as f:
            json.dump(data, f, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
    except Exception as e:
        print(f"Cache save error: {e}")

def load_cache_from_file(video_id: str, method: str, frame_idx: int) -> Dict:
    """파일에서 캐시 로드"""
    cache_dir = get_cache_dir()
    filename = get_cache_filename(video_id, method, frame_idx)
    cache_file = cache_dir / filename
    
    if cache_file.exists():
        try:
            with open(cache_file, 'r') as f:
                data = json.load(f)
            # numpy 배열로 복원
            for key, value in data.items():
                if isinstance(value, list) and len(value) > 0 and isinstance(value[0], list):
                    data[key] = np.array(value)
            return data
        except Exception as e:
            print(f"Cache load error: {e}")
            return None
    return None

def clear_cache_files():
    """모든 캐시 파일 삭제"""
    cache_dir = get_cache_dir()
    deleted_count = 0
    for cache_file in cache_dir.glob("*.json"):
        try:
            cache_file.unlink()
            deleted_count += 1
        except Exception as e:
            print(f"Failed to delete {cache_file}: {e}")
    return deleted_count

## 인터랙티브 GUI

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import base64
from io import BytesIO

import sys
import warnings
from contextlib import redirect_stdout, redirect_stderr
from io import StringIO

warnings.filterwarnings('ignore')

# 디버그 출력용 위젯
debug_output = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='150px',
        border='1px solid #ccc',
        overflow='auto'
    )
)

if not hasattr(sys.modules[__name__], '_GLOBAL_GUI_STATE_V2'):
    class GlobalGUIStateV2:
        def __init__(self):
            self.gui_instance = None
            self.debug_messages = set()
            self.debug_initialized = False
            
    sys.modules[__name__]._GLOBAL_GUI_STATE_V2 = GlobalGUIStateV2()

_global_state_v2 = sys.modules[__name__]._GLOBAL_GUI_STATE_V2

def debug_print_v2(message):
    """완전 중복 방지 디버그 출력 V2"""
    if message in _global_state_v2.debug_messages:
        return
    
    _global_state_v2.debug_messages.add(message)
    
    from datetime import datetime
    timestamp = datetime.now().strftime("%H:%M:%S")
    debug_output.append_stdout(f"[{timestamp}] {message}\n")

class FixedKeypointComparisonGUI:
    def __init__(self):
        if hasattr(self, '_initialized'):
            return
        
        self.current_video_id = available_videos[0] if available_videos else None
        self.current_frame_idx = 0
        self.keypoint_files = []
        self.video_file = None
        self.extractor = None
        self.current_method = "openpose"
        
        self.init_extractors()
        self.create_widgets()
        self.load_video_data()
        self._initialized = True
        
    def init_extractors(self):
        """키포인트 추출기 초기화"""
        self.available_methods = []
        
        if OPENPOSE_AVAILABLE:
            try:
                with redirect_stdout(StringIO()), redirect_stderr(StringIO()):
                    self.openpose_extractor = KeypointExtractor(method="openpose")
                self.available_methods.append("openpose")
                debug_print_v2("✅ OpenPose extractor initialized")
            except Exception as e:
                debug_print_v2(f"❌ Failed to initialize OpenPose: {e}")
        
        if MEDIAPIPE_AVAILABLE:
            try:
                self.mediapipe_extractor = KeypointExtractor(method="mediapipe")
                self.available_methods.append("mediapipe")
                debug_print_v2("✅ MediaPipe extractor initialized")
            except Exception as e:
                debug_print_v2(f"❌ Failed to initialize MediaPipe: {e}")
        
        # 기본 추출기 설정
        if "openpose" in self.available_methods:
            self.extractor = self.openpose_extractor
            self.current_method = "openpose"
        elif "mediapipe" in self.available_methods:
            self.extractor = self.mediapipe_extractor
            self.current_method = "mediapipe"
        
    def create_widgets(self):
        """GUI 위젯 생성"""
        self.video_dropdown = widgets.Dropdown(
            options=available_videos,
            value=self.current_video_id,
            description='🎬 Video:',
            style={'description_width': 'initial'}
        )
        self.video_dropdown.observe(self.on_video_change, names='value')
        
        self.method_dropdown = widgets.Dropdown(
            options=self.available_methods,
            value=self.current_method if self.current_method in self.available_methods else None,
            description='🔧 Method:',
            style={'description_width': 'initial'}
        )
        self.method_dropdown.observe(self.on_method_change, names='value')
        
        self.frame_label = widgets.Label(value="Frame: 0 / 0")
        self.status_label = widgets.HTML(value="<div style='color: blue;'>📍 Ready</div>")
        
        # 기본 버튼들
        self.prev_btn = widgets.Button(description='◀ Prev', button_style='info', layout=widgets.Layout(width='80px'))
        self.next_btn = widgets.Button(description='Next ▶', button_style='info', layout=widgets.Layout(width='80px'))
        self.extract_btn = widgets.Button(description='🔍 Extract', button_style='warning', layout=widgets.Layout(width='80px'))
        
        # 구간 분석 위젯들
        self.start_frame_input = widgets.IntText(value=0, description='Start:', layout=widgets.Layout(width='120px'))
        self.end_frame_input = widgets.IntText(value=9, description='End:', layout=widgets.Layout(width='120px'))
        self.interval_btn = widgets.Button(description='📊 구간분석', button_style='success', layout=widgets.Layout(width='100px'))
        
        # 캐시 관리 버튼
        self.clear_cache_btn = widgets.Button(description='🗑️ 캐시삭제', button_style='danger', layout=widgets.Layout(width='100px'))
        
        # 이벤트 핸들러 등록
        self.prev_btn.on_click(self.prev_frame)
        self.next_btn.on_click(self.next_frame)
        self.extract_btn.on_click(self.extract_current_frame)
        self.interval_btn.on_click(self.analyze_interval)
        self.clear_cache_btn.on_click(self.clear_cache)
        
        self.stats_output = widgets.HTML(value="통계 정보가 여기에 표시됩니다.")
        self.image_output = widgets.HTML(value="이미지가 여기에 표시됩니다.")  # HTML로 변경
        
        # 레이아웃
        frame_controls = widgets.HBox([self.prev_btn, self.frame_label, self.next_btn, self.extract_btn])
        interval_controls = widgets.HBox([
            widgets.Label('구간 분석:'), 
            self.start_frame_input, 
            self.end_frame_input, 
            self.interval_btn,
            self.clear_cache_btn
        ])
        
        self.main_widget = widgets.VBox([
            widgets.HTML("<h2>🎯 Fixed Keypoint Comparison Tool</h2>"),
            self.video_dropdown,
            self.method_dropdown,
            frame_controls,
            interval_controls,
            self.status_label,
            widgets.HBox([self.image_output, self.stats_output]),
        ])
        
    def clear_cache(self, btn):
        """파일 캐시 삭제"""
        deleted_count = clear_cache_files()
        self.status_label.value = f"<div style='color: green;'>🗑️ {deleted_count}개 캐시 파일이 삭제되었습니다.</div>"
        debug_print_v2(f"🗑️ Deleted {deleted_count} cache files")
        
    def figure_to_html(self, fig):
        """matplotlib figure를 HTML img 태그로 변환"""
        buffer = BytesIO()
        fig.savefig(buffer, format='png', bbox_inches='tight', dpi=100)
        buffer.seek(0)
        image_png = buffer.getvalue()
        buffer.close()
        
        # base64 인코딩
        graphic = base64.b64encode(image_png)
        graphic = graphic.decode('utf-8')
        
        # figure 닫기
        plt.close(fig)
        
        return f'<img src="data:image/png;base64,{graphic}" style="max-width: 100%; height: auto;"/>'
        
    def on_method_change(self, change):
        """키포인트 추출 방법 변경"""
        new_method = change['new']
        self.current_method = new_method
        
        if new_method == "mediapipe" and hasattr(self, 'mediapipe_extractor'):
            self.extractor = self.mediapipe_extractor
        elif new_method == "openpose" and hasattr(self, 'openpose_extractor'):
            self.extractor = self.openpose_extractor
        
        self.update_frame_label()
        self.status_label.value = f"<div style='color: blue;'>🔧 Switched to {new_method.upper()}</div>"
        debug_print_v2(f"🔧 Switched to {new_method.upper()}")
        
    def load_video_data(self):
        """비디오 데이터 로드"""
        if not self.current_video_id:
            return
            
        self.status_label.value = "<div style='color: orange;'>📁 Loading video data...</div>"
        
        keypoint_dir_name = self.current_video_id.replace('.mp4', '')
        keypoint_dir = keypoint_base_path / keypoint_dir_name
        
        debug_print_v2(f"🔍 Loading data for: {self.current_video_id}")
        
        if keypoint_dir.exists():
            self.keypoint_files = sorted(list(keypoint_dir.glob('*_keypoints.json')))
        else:
            self.keypoint_files = []
            
        self.video_file = video_base_path / self.current_video_id
        debug_print_v2(f"📊 Found {len(self.keypoint_files)} keypoint files")
            
        self.current_frame_idx = 0
        
        # 구간 분석 입력 범위 업데이트
        max_frame = len(self.keypoint_files) - 1
        self.start_frame_input.max = max_frame
        self.end_frame_input.max = max_frame
        self.end_frame_input.value = min(9, max_frame)
        
        self.update_frame_label()
        self.update_visualization()  # 초기 시각화
        
        self.status_label.value = f"<div style='color: green;'>✅ Loaded {len(self.keypoint_files)} frames</div>"
        
    def on_video_change(self, change):
        """비디오 선택 변경"""
        self.current_video_id = change['new']
        self.load_video_data()
        
    def prev_frame(self, btn):
        """이전 프레임"""
        if self.current_frame_idx > 0:
            debug_print_v2(f"🔙 Prev button clicked. Current: {self.current_frame_idx}")
            
            self.current_frame_idx -= 1
            self.update_frame_label()
            self.update_visualization()
            
            self.status_label.value = f"<div style='color: blue;'>◀ Frame {self.current_frame_idx + 1}</div>"
            debug_print_v2(f"✅ Moved to frame {self.current_frame_idx + 1}")
            
    def next_frame(self, btn):
        """다음 프레임"""
        if self.current_frame_idx < len(self.keypoint_files) - 1:
            debug_print_v2(f"🔜 Next button clicked. Current: {self.current_frame_idx}")
            
            self.current_frame_idx += 1
            self.update_frame_label()
            self.update_visualization()
            
            self.status_label.value = f"<div style='color: blue;'>▶ Frame {self.current_frame_idx + 1}</div>"
            debug_print_v2(f"✅ Moved to frame {self.current_frame_idx + 1}")
    
    def get_cached_extraction(self, frame_idx: int):
        """파일에서 캐시된 추출 결과 가져오기"""
        cached_data = load_cache_from_file(self.current_video_id, self.current_method, frame_idx)
        if cached_data:
            debug_print_v2(f"💾 Using cached extraction for frame {frame_idx}")
            return cached_data
        return None
    
    def cache_extraction(self, frame_idx: int, extracted_coords: Dict):
        """추출 결과를 파일로 캐싱"""
        save_cache_to_file(self.current_video_id, self.current_method, frame_idx, extracted_coords)
        debug_print_v2(f"💾 Cached extraction for frame {frame_idx}")
            
    def extract_current_frame(self, btn):
        """키포인트 추출 - 파일 캐싱 기능 포함"""
        if not self.video_file or not self.extractor:
            self.status_label.value = "<div style='color: red;'>❌ Video file or extractor not available</div>"
            return
        
        # 파일 캐시 확인
        cached_result = self.get_cached_extraction(self.current_frame_idx)
        if cached_result is not None:
            self.update_visualization_with_extraction(cached_result)
            self.status_label.value = f"<div style='color: green;'>💾 Used cached data</div>"
            return
            
        self.status_label.value = f"<div style='color: orange;'>🔍 Extracting keypoints...</div>"
        self.extract_btn.disabled = True
        
        debug_print_v2(f"🔍 Starting extraction for frame {self.current_frame_idx}")
        
        try:
            # 프레임 가져오기
            frame = get_frame_from_video(self.video_file, self.current_frame_idx, debug_print_v2)
            if frame is None:
                self.status_label.value = "<div style='color: red;'>❌ Failed to get frame</div>"
                debug_print_v2("❌ Failed to get frame")
                return
                
            # 키포인트 추출 (출력 차단)
            self.status_label.value = f"<div style='color: orange;'>⚙️ Processing...</div>"
            debug_print_v2(f"⚙️ Processing with {self.current_method.upper()}")
            
            with redirect_stdout(StringIO()), redirect_stderr(StringIO()):
                keypoint_data = extract_single_frame_keypoints(frame, self.extractor)
                
            extracted_coords = extract_coordinates(keypoint_data)
            
            # 파일로 캐싱
            self.cache_extraction(self.current_frame_idx, extracted_coords)
            
            self.status_label.value = f"<div style='color: green;'>✅ Extraction complete!</div>"
            debug_print_v2("✅ Extraction completed")
            
            # 시각화 업데이트
            self.update_visualization_with_extraction(extracted_coords)
            
        except Exception as e:
            self.status_label.value = f"<div style='color: red;'>❌ Error during extraction</div>"
            debug_print_v2(f"❌ Error: {str(e)}")
        finally:
            self.extract_btn.disabled = False
    
    def analyze_interval(self, btn):
        """구간 분석 수행 - 파일 캐시 활용"""
        start_frame = self.start_frame_input.value
        end_frame = self.end_frame_input.value
        
        if start_frame > end_frame:
            self.status_label.value = "<div style='color: red;'>❌ Start frame must be <= End frame</div>"
            return
        
        if end_frame >= len(self.keypoint_files):
            self.status_label.value = f"<div style='color: red;'>❌ End frame exceeds max frame ({len(self.keypoint_files)-1})</div>"
            return
        
        self.status_label.value = f"<div style='color: orange;'>📊 Analyzing interval {start_frame+1}-{end_frame+1}...</div>"
        self.interval_btn.disabled = True
        
        try:
            all_part_stats = []
            
            for frame_idx in range(start_frame, end_frame + 1):
                # 원본 키포인트 로드
                orig_data = load_keypoint_data(self.keypoint_files[frame_idx])
                orig_coords = extract_coordinates(orig_data)
                
                # 파일 캐시에서 추출된 키포인트 확인
                extracted_coords = self.get_cached_extraction(frame_idx)
                
                if extracted_coords is None:
                    # 캐시에 없으면 추출
                    frame = get_frame_from_video(self.video_file, frame_idx)
                    if frame is not None:
                        with redirect_stdout(StringIO()), redirect_stderr(StringIO()):
                            keypoint_data = extract_single_frame_keypoints(frame, self.extractor)
                        extracted_coords = extract_coordinates(keypoint_data)
                        self.cache_extraction(frame_idx, extracted_coords)
                    else:
                        extracted_coords = {}
                
                # 파트별 통계 계산
                if extracted_coords:
                    part_stats = calculate_part_stats(orig_coords, extracted_coords)
                    all_part_stats.append(part_stats)
                
                # 진행 상황 업데이트
                progress = (frame_idx - start_frame + 1) / (end_frame - start_frame + 1) * 100
                self.status_label.value = f"<div style='color: orange;'>📊 Processing... {progress:.1f}%</div>"
            
            # 구간 통계 계산
            interval_stats = calculate_interval_stats(all_part_stats)
            
            # 결과 표시
            stats_html = format_interval_stats_html(interval_stats, start_frame, end_frame)
            self.stats_output.value = stats_html
            
            self.status_label.value = f"<div style='color: green;'>✅ Interval analysis complete!</div>"
            debug_print_v2(f"📊 Interval analysis completed for frames {start_frame}-{end_frame}")
            
        except Exception as e:
            self.status_label.value = f"<div style='color: red;'>❌ Error during interval analysis</div>"
            debug_print_v2(f"❌ Interval analysis error: {str(e)}")
        finally:
            self.interval_btn.disabled = False
        
    def update_frame_label(self):
        """프레임 라벨 업데이트 - 캐시 상태 표시"""
        total = len(self.keypoint_files)
        current = self.current_frame_idx + 1 if total > 0 else 0
        
        # 파일 캐시 상태 확인
        cached_data = load_cache_from_file(self.current_video_id, self.current_method, self.current_frame_idx)
        cache_status = "💾" if cached_data else "🔍"
        
        self.frame_label.value = f"Frame: {current} / {total} {cache_status}"
        
        self.prev_btn.disabled = (self.current_frame_idx <= 0)
        self.next_btn.disabled = (self.current_frame_idx >= total - 1)
        self.extract_btn.disabled = (not self.video_file or not self.extractor)
    
    def update_visualization_with_extraction(self, extracted_coords: Dict):
        """추출된 데이터로 시각화 업데이트"""
        if not self.keypoint_files or self.current_frame_idx >= len(self.keypoint_files):
            return
            
        # 원본 키포인트 로드
        orig_data = load_keypoint_data(self.keypoint_files[self.current_frame_idx])
        orig_coords = extract_coordinates(orig_data)
        
        # 비디오 프레임 가져오기
        frame = get_frame_from_video(self.video_file, self.current_frame_idx)
        if frame is None:
            frame = np.zeros((480, 640, 3), dtype=np.uint8)
            
        # 시각화 생성 및 HTML 변환
        html_img = self.create_comparison_plot(frame, orig_coords, extracted_coords)
        self.image_output.value = html_img
            
        # 파트별 통계 업데이트
        if extracted_coords:
            part_stats = calculate_part_stats(orig_coords, extracted_coords)
            stats_html = format_part_stats_html(part_stats)
            method_info = f"<div style='background: #f0f0f0; padding: 5px; margin-bottom: 10px;'><b>🔧 Method:</b> {self.current_method.upper()}</div>"
            self.stats_output.value = method_info + stats_html
        else:
            method_info = f"<div style='background: #f0f0f0; padding: 5px;'><b>🔧 Method:</b> {self.current_method.upper()}</div>"
            self.stats_output.value = method_info + "<div style='padding: 10px;'>🔍 Extract 버튼을 눌러 키포인트를 추출하세요.</div>"
        
    def update_visualization(self):
        """시각화 업데이트 - 파일 캐시된 데이터 우선 사용"""
        if not self.keypoint_files or self.current_frame_idx >= len(self.keypoint_files):
            self.image_output.value = "<div style='padding: 20px; text-align: center;'>❌ No keypoint data available</div>"
            return
        
        # 파일 캐시된 추출 데이터 확인
        extracted_coords = self.get_cached_extraction(self.current_frame_idx)
        
        if extracted_coords is not None:
            self.update_visualization_with_extraction(extracted_coords)
        else:
            # 캐시에 없으면 원본만 표시
            orig_data = load_keypoint_data(self.keypoint_files[self.current_frame_idx])
            orig_coords = extract_coordinates(orig_data)
            
            frame = get_frame_from_video(self.video_file, self.current_frame_idx)
            if frame is None:
                frame = np.zeros((480, 640, 3), dtype=np.uint8)
                
            # 원본만 표시
            html_img = self.create_comparison_plot(frame, orig_coords, None)
            self.image_output.value = html_img
                
            method_info = f"<div style='background: #f0f0f0; padding: 5px;'><b>🔧 Method:</b> {self.current_method.upper()}</div>"
            self.stats_output.value = method_info + "<div style='padding: 10px;'>🔍 Extract 버튼을 눌러 키포인트를 추출하세요.</div>"
                
    def create_comparison_plot(self, frame: np.ndarray, original_coords: Dict, extracted_coords: Dict = None):
        """키포인트 비교 plot 생성 후 HTML 반환"""
        colors = {
            'pose': {'orig': 'red', 'extr': 'blue'},
            'face': {'orig': 'green', 'extr': 'cyan'},
            'left_hand': {'orig': 'orange', 'extr': 'yellow'},
            'right_hand': {'orig': 'purple', 'extr': 'pink'}
        }
        
        # 새 figure 생성
        if extracted_coords:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        else:
            fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))
        
        # 원본 키포인트
        ax1.imshow(frame)
        ax1.set_title(f"Original Keypoints\\nFrame {self.current_frame_idx + 1}", fontsize=14)
        
        for part_name, coords in original_coords.items():
            if part_name in colors:
                valid_coords = coords[np.any(coords != 0, axis=1)]
                if len(valid_coords) > 0:
                    ax1.scatter(valid_coords[:, 0], valid_coords[:, 1], 
                               c=colors[part_name]['orig'], s=25, alpha=0.8, 
                               label=f'{part_name.replace("_", " ").title()}')
        
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.axis('off')
        
        # 비교 (추출된 데이터가 있는 경우)
        if extracted_coords:
            ax2.imshow(frame)
            ax2.set_title(f"Original vs {self.current_method.upper()}", fontsize=14)
            
            # 원본 키포인트
            for part_name, coords in original_coords.items():
                if part_name in colors:
                    valid_coords = coords[np.any(coords != 0, axis=1)]
                    if len(valid_coords) > 0:
                        ax2.scatter(valid_coords[:, 0], valid_coords[:, 1], 
                                   c=colors[part_name]['orig'], s=30, alpha=0.8, 
                                   label=f'{part_name.replace("_", " ").title()} (orig)', 
                                   marker='o')
            
            # 추출된 키포인트
            for part_name, coords in extracted_coords.items():
                if part_name in colors:
                    valid_coords = coords[np.any(coords != 0, axis=1)]
                    if len(valid_coords) > 0:
                        ax2.scatter(valid_coords[:, 0], valid_coords[:, 1], 
                                   c=colors[part_name]['extr'], s=25, alpha=0.7, 
                                   label=f'{part_name.replace("_", " ").title()} ({self.current_method})', 
                                   marker='x')
            
            ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax2.axis('off')
        
        plt.tight_layout()
        
        # HTML로 변환
        html_img = self.figure_to_html(fig)
        
        debug_print_v2(f"🎯 Figure created and converted to HTML")
        
        return html_img
        
    def display(self):
        return self.main_widget

def create_fixed_gui():
    # 기존 GUI 인스턴스가 있다면 정리
    if _global_state_v2.gui_instance is not None:
        debug_print_v2("🔄 Replacing existing GUI instance")
    
    if available_videos:
        if not _global_state_v2.debug_initialized:
            debug_print_v2("🚀 Fixed system initialization")
            _global_state_v2.debug_initialized = True
        
        _global_state_v2.gui_instance = FixedKeypointComparisonGUI()
        debug_print_v2("🎯 Fixed GUI Created successfully!")
        
        display(debug_output)
        display(_global_state_v2.gui_instance.display())
    else:
        print("❌ No videos available")

create_fixed_gui()